In [1]:
import pandas as pd
from keras.models import load_model
import tensorflow as tf
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

2023-08-09 17:20:13.399866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv','chmom',
             'chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa','ill','indmom','invest',
             'lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr','orgcap','pchgm_pchsale','pchsale_pchinvt',
             'pchsale_pchrect','pchsale_pchxsga','retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp',
             'std_dolvol','std_turn','turn']

core = ['date','permno','ticker','comnam','exchcd','exchname','siccd',
        'indname','size_class','mve_m','ret_adj_ex',]

#core = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
#        'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

print(len(char_core), len(core))

data = pd.read_pickle('norm_df49.pkl')
data['date'] = pd.to_datetime(data.index).to_period('M')
data.sort_values(by='date', ascending=True, inplace=True)

X = data[char_core+['date']]
X_info = data[core]
Y = data[['predicted_return','date']]
print(X.shape, Y.shape)

49 11
(210205, 50) (210205, 2)


In [31]:
X_info = data[core]
x_test_info = X_info[(X_info['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

stock_list = x_test_info['permno'].unique()
month_list = x_test_info.index.unique()
print(f"Total Number of stocks: {len(stock_list)}")
print(f"Total Number of Months: {len(month_list)}")

Total Number of stocks: 607
Total Number of Months: 269


In [38]:
######################################
### Scenario 1: Linear Regression ####
######################################


x_train1 = X[(X['date'] < pd.Period((str(2000)+"-1"),freq='M'))]
y_train1 = Y[(Y['date'] < pd.Period((str(2000)+"-1"),freq='M'))]

x_test1 = X[(X['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]
y_test1 = Y[(Y['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

x_test1 = x_test1.drop("date", axis=1)
y_test1 = y_test1.drop("date", axis=1)

x_train1 = x_train1.drop("date", axis=1)
y_train1 = y_train1.drop("date", axis=1)

# Linear Model Training 
regr = linear_model.LinearRegression()
regr.fit(x_train1, y_train1)
yhat_linear_1 = regr.predict(x_test1)

# Calculate MSE of Linear Regression 
mse = mean_squared_error(y_test1, yhat_linear_1)
print(f"【Scenario1】MSE of linear Regression:",mse)

# Calculate R2 of Linear REgression 
a = np.square(y_test1.values.reshape(-1) - yhat_linear_1.reshape(-1)).sum()
b = np.square(y_test1.values.reshape(-1)).sum()
1-a/b
print(f"【Scenario1】R^2 of linear Regression:",1-a/b)

# Save y_real, y_hat into DataFrame 
x_test_info['yhat_linear_1'] = yhat_linear_1
x_test_info['y_real'] = y_test1.values.reshape(-1)

【Scenario1】MSE of linear Regression: 0.01566968182931832
【Scenario1】R^2 of linear Regression: 0.005432492063096261


/var/folders/lj/d6lmnkt148jf782ksr_lmtnm0000gn/T/ipykernel_39313/1143754786.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_info['yhat_linear_1'] = yhat_linear_1
/var/folders/lj/d6lmnkt148jf782ksr_lmtnm0000gn/T/ipykernel_39313/1143754786.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_info['y_real'] = y_test1.values.reshape(-1)


In [62]:
def metricsEV(month_list, result_df, yhat_name):
    """
    For each month:
    1> square errors 
    2> get mean value 
    3> average of all months 
    """
    r2_stock = {}
    num_ = []
    deno_ = []
    for i in month_list:
        df = result_df[result_df['date'] == str(i)]
        #num_stock = df.shape[0]
        num = np.mean(np.square(df['y_real'] -  df[yhat_name]))#.sum() / num_stock
        deno = np.mean(np.square(df['y_real']))#.sum() /num_stock
        num_.append(num)
        deno_.append(deno)
    return 1 - np.mean(num_) / np.mean(deno_)

In [74]:
def metricsXS_R2_weighted(stock_list, result_df, yhat_name):
    """
    For each stock:
    1> square errors 
    2> get mean value 
    3> average of all months 
    """
    r2_stock = {}
    num_ = []
    deno_ = []
    for i in stock_list:
        df = result_df[result_df['permno'] == i]
        num_month = df.shape[0]
        num = np.square(np.mean(df['y_real'] -  df[yhat_name])) * num_month
        deno = np.square(np.mean(df[yhat_name]))  * num_month
        num_.append(num)
        deno_.append(deno)
    return 1 - np.mean(num_) / np.mean(deno_)

In [81]:
def metricsXS_R2_NOweighted(stock_list, result_df, yhat_name):
    """
    For each stock:
    1> square errors 
    2> get mean value 
    3> average of all months 
    """
    r2_stock = {}
    num_ = []
    deno_ = []
    for i in stock_list:
        df = result_df[result_df['permno'] == i]
        num_month = df.shape[0]
        num = np.square(np.mean(df['y_real'] -  df[yhat_name]))
        deno = np.square(np.mean(df[yhat_name]))
        num_.append(num)
        deno_.append(deno)
    return 1 - np.mean(num_) / np.mean(deno_)

In [64]:
ev_linear1 = metricsEV(month_list, result_df=x_test_info, yhat_name="yhat_linear_1")
print("【Scenario 1】 EV of Linear Model: ",ev_linear1)

【Scenario 1】 EV of Linear Model:  0.005101629835022381


In [75]:
xs_R2_weighted_linear1 = metricsXS_R2_weighted(stock_list, result_df=x_test_info, yhat_name="yhat_linear_1")
print(f"【Scenario 1】XS-R^2 of Linear Regression (weighted):",xs_R2_weighted_linear1)

【Scenario 1】XS-R^2 of Linear Regression (weighted): -1.6860012050668498


In [82]:
xs_R2_NOweighted_linear1 = metricsXS_R2_NOweighted(stock_list, result_df=x_test_info, yhat_name="yhat_linear_1")
print(f"【Scenario 1】XS-R^2 of Linear Regression (no weighted):",xs_R2_NOweighted_linear1)

【Scenario 1】XS-R^2 of Linear Regression (no weighted): -30.426142440845403


---

## Scenario II Linear Regression

In [83]:
# Linear Model of Senario 2
y_hat_linear = []
mse_linear = []
r2_linear = []

for i in range(2000, 2023):
    print(f"Year {i}")
    x_train = X[ (X['date'] < pd.Period((str(i)+"-1"),freq='M')) ]
    y_train = Y[ (Y['date'] < pd.Period((str(i)+"-1"),freq='M')) ]

    x_test = X[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]
    y_test = Y[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]

    x_train = x_train.drop("date", axis=1)
    y_train = y_train.drop("date", axis=1)
    x_test = x_test.drop("date", axis=1)
    y_test = y_test.drop("date", axis=1)

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)
    y_pred_linear = regr.predict(x_test)
    mse = mean_squared_error(y_test, y_pred_linear)
    for a in y_pred_linear.reshape(-1):
        y_hat_linear.append(a)

Year 2000
Year 2001
Year 2002
Year 2003
Year 2004
Year 2005
Year 2006
Year 2007
Year 2008
Year 2009
Year 2010
Year 2011
Year 2012
Year 2013
Year 2014
Year 2015
Year 2016
Year 2017
Year 2018
Year 2019
Year 2020
Year 2021
Year 2022


In [89]:
# Save y_real, y_hat into DataFrame "Scenario II"
x_test_info['yhat_linear_2'] = y_hat_linear

/var/folders/lj/d6lmnkt148jf782ksr_lmtnm0000gn/T/ipykernel_39313/248754443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_info['yhat_linear_2'] = y_hat_linear


In [96]:
# Calculate MSE of Linear Regression 
mse = mean_squared_error(x_test_info['y_real'], x_test_info['yhat_linear_2'])
print(f"【Scenario2】MSE of linear Regression:",mse)


# Calculate R2 of Linear REgression 
a = np.square(x_test_info['y_real']- x_test_info['yhat_linear_2']).sum()
b = np.square(x_test_info['y_real']).sum()
1-a/b
print(f"【Scenario2】R^2 of linear Regression:",1-a/b)


# Calculae EV of Linear Regression 
ev_linear2 = metricsEV(month_list, result_df=x_test_info, yhat_name="yhat_linear_2")
print("【Scenario 2】 EV of Linear Model: ",ev_linear2)


# Calculate XS-R2 (weighted) of Linear Regression 
xs_R2_weighted_linear2 = metricsXS_R2_weighted(stock_list, result_df=x_test_info, yhat_name="yhat_linear_2")
print(f"【Scenario 2】XS-R^2 of Linear Regression (weighted):",xs_R2_weighted_linear2)


# Calculate XS-R2 (no weighted) of Linear Regression 
xs_R2_NOweighted_linear2 = metricsXS_R2_NOweighted(stock_list, result_df=x_test_info, yhat_name="yhat_linear_2")
print(f"【Scenario 2】XS-R^2 of Linear Regression (no weighted):",xs_R2_NOweighted_linear2)

【Scenario2】MSE of linear Regression: 0.01566540810352375
【Scenario2】R^2 of linear Regression: 0.0057037489309388345
【Scenario 2】 EV of Linear Model:  0.005339982538576593
【Scenario 2】XS-R^2 of Linear Regression (weighted): -0.9542138628254369
【Scenario 2】XS-R^2 of Linear Regression (no weighted): -23.84399135612912


---

## Scenario III Linear Regression 

In [126]:
Y_2 = Y[Y['date'] >= pd.Period((str(1999)+"-"+str(12)),freq='M')]
X_2 = X[X['date'] >= pd.Period((str(1999)+"-"+str(12)),freq='M')]
y_hat_lin = [] 
y_real = []

for i in range(len(Y_2['date'].unique())):
    train_date = Y_2['date'].unique()[i]
    if train_date == pd.Period((str(2022)+"-"+str(5)),freq='M'):
        break
    test_date = Y_2['date'].unique()[i+1]
    x_train = X[X["date"] == train_date ]
    x_train=x_train.drop("date", axis=1)
    y_train = Y[Y["date"] == train_date ]
    y_train= y_train.drop("date", axis=1)
    x_test = X[X["date"] == test_date ]
    x_test=x_test.drop("date", axis=1)
    y_test = Y[Y["date"] == test_date ]
    y_test=y_test.drop("date", axis=1)
    for a in y_test.values.reshape(-1):
        y_real.append(a)

    regr = linear_model.LinearRegression()
    regr.fit(x_train.values, y_train.values)
    yhat_lin = regr.predict(x_test.values).reshape(-1)

    for c in yhat_lin:
        y_hat_lin.append(c)
        
# Save y_real, y_hat into DataFrame "Scenario III"
x_test_info['yhat_linear_3'] = y_hat_lin

/var/folders/lj/d6lmnkt148jf782ksr_lmtnm0000gn/T/ipykernel_39313/3647538270.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_info['yhat_linear_3'] = y_hat_lin


In [135]:
# Calculate MSE of Linear Regression 
mse = mean_squared_error(x_test_info['y_real'], x_test_info['yhat_linear_3'])
print(f"【Scenario 3】MSE of linear Regression:",mse)


# Calculate R2 of Linear REgression 
a = np.square(x_test_info['y_real']- x_test_info['yhat_linear_3']).sum()
b = np.square(x_test_info['y_real']).sum()
1-a/b
print(f"【Scenario 3】R^2 of linear Regression:",1-a/b)


# Calculae EV of Linear Regression 
ev_linear3 = metricsEV(month_list, result_df=x_test_info, yhat_name="yhat_linear_3")
print("【Scenario 3】 EV of Linear Model: ",ev_linear3)


# Calculate XS-R2 (weighted) of Linear Regression 
xs_R2_weighted_linear3 = metricsXS_R2_weighted(stock_list, result_df=x_test_info, yhat_name="yhat_linear_3")
print(f"【Scenario 3】XS-R^2 of Linear Regression (weighted):",xs_R2_weighted_linear3)


# Calculate XS-R2 (no weighted) of Linear Regression 
xs_R2_NOweighted_linear3 = metricsXS_R2_NOweighted(stock_list, result_df=x_test_info, yhat_name="yhat_linear_3")
print(f"【Scenario 3】XS-R^2 of Linear Regression (no weighted):",xs_R2_NOweighted_linear3)

【Scenario 3】MSE of linear Regression: 0.02179186765218634
【Scenario 3】R^2 of linear Regression: -0.3831476439792305
【Scenario 3】 EV of Linear Model:  -0.4083205724696244
【Scenario 3】XS-R^2 of Linear Regression (weighted): 0.24162446917706693
【Scenario 3】XS-R^2 of Linear Regression (no weighted): -1.7757749704850103


---

In [137]:
# Save Three scenarios Linear Regression Result into DataFrame 
x_test_info.to_pickle("x_test_info.pkl")